In [ ]:
!pip install imblearn shap

In [15]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.calibration import LabelEncoder
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.inspection import permutation_importance
from sklearn.discriminant_analysis import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
shap.initjs()

In [17]:
# Hyperparameters for various functions
seed = 0
smote_sample_strategy = 0.4
random_under_sample_strategy = 0.9
smote_k_neighbors = 5

In [ ]:
root_dir = os.path.dirname(os.getcwd())
df = pd.read_csv(os.path.join(root_dir, "data/processed/data_cleaned.csv"))

In [ ]:
df.head()

In [ ]:
df.info()

In [21]:
def create_metrics(df):
    df_temp = df.copy()
    grouping_vars = ['subject','session']
    df_temp["dx"] = df_temp.groupby(grouping_vars)["x_coordinate"].diff()
    df_temp["dy"] = df_temp.groupby(grouping_vars)["y_coordinate"].diff()
    df_temp["delta_altitude"] = df_temp.groupby(grouping_vars)["altitude"].diff().abs()
    df_temp["delta_pressure"] = df_temp.groupby(grouping_vars)["pressure"].diff().abs()
    df_temp["delta_azimuth"] = df_temp.groupby(grouping_vars)["azimuth"].diff().abs()
    # Calculate the delta distance as the Euclidean distance for each group
    df_temp["distance"] = np.sqrt(df_temp["dx"] ** 2 + df_temp["dy"] ** 2)
    # Compute time difference in seconds
    df_temp["dt"] = pd.to_datetime(df_temp["date"]).diff().dt.total_seconds()
    # Calculate speed
    df_temp["speed"] = df_temp["distance"] / df_temp["dt"]

    df_temp = df_temp.groupby(['session','control']).agg({
            "altitude":'mean',
            "pressure":'mean',
            "azimuth":'mean',        
            "delta_altitude":'mean',
            "delta_pressure":'mean',
            "delta_azimuth":'mean',
            "speed":'mean',
        }).reset_index()
    df_temp = df_temp.drop("session", axis=1)
    return df_temp
aggregated_data = create_metrics(df)

In [ ]:
aggregated_data.head()

In [23]:
X = aggregated_data.drop(["control"], axis=1)
y = aggregated_data["control"].copy()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=seed)

In [ ]:
%%time
def train_rf(perform_fs=False):
    pipeline = Pipeline([])#imbPipeline([])
    if perform_fs:
        pipeline.steps.append(
                ("selector", SelectKBest(mutual_info_classif, k=4))
            )
        # pipeline.steps.append(
        #         ("selector", SequentialFeatureSelector(KNeighborsClassifier(n_neighbors=3),
        #                                                                 n_features_to_select=4,
        #                                                                 direction='forward')),
        #     )
    pipeline.steps.extend([
        ("scaler", StandardScaler()),
        # ("smote", SMOTE(sampling_strategy=smote_sample_strategy, k_neighbors=smote_k_neighbors, random_state=seed)),
        # ("undersampler", RandomUnderSampler(sampling_strategy=random_under_sample_strategy, random_state=seed)),
        ("classifier", RandomForestClassifier(random_state=0))
    ])
    pipeline.fit(X_train, y_train)
    train_accuracy = pipeline.score(X_train, y_train)
    test_accuracy = pipeline.score(X_test, y_test)
    print(f"Train accuracy: {train_accuracy*100}%, Train accuracy: {test_accuracy*100}%")
    return pipeline
rf_pipeline = train_rf(False)
rf_model = rf_pipeline["classifier"]
feature_importances = rf_model.feature_importances_

In [ ]:
numerical_features = X_train.columns.copy()
sorted(zip(list(feature_importances), numerical_features), reverse=True) 

In [ ]:
# This snippet of code was taken from: https://www.rasgoml.com/feature-engineering-tutorials/how-to-generate-feature-importance-plots-from-scikit-learn
sorted_idx = np.argsort(feature_importances)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importances[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Feature Importance')

In [139]:
explainer = shap.TreeExplainer(rf_model, feature_names=numerical_features)

In [122]:
feature_values = rf_pipeline.named_steps["scaler"].transform(X_train)
explanation = explainer(feature_values)
shap_values = explainer.shap_values(feature_values)

In [ ]:
explainer.expected_value

In [ ]:
shap.plots.beeswarm(explanation[:,:,1], show=True)

In [ ]:
explainer.expected_value[1]

In [149]:
id = 0

In [ ]:
shap.plots.force(explainer.expected_value[1], shap_values[id][:,1], X_train.iloc[id:id+1])

In [ ]:
shap.plots.waterfall(explanation[0,:,1])

In [ ]:
shap.summary_plot(shap_values=shap_values[..., 1], features=X_test, plot_type='bar', max_display=10, plot_size=0.25)